## Función `AI_AGG`

La función `AI_AGG` reduce una columna de datos de texto mediante una tarea descrita en lenguaje natural. Esta función no está sujeta a las limitaciones de la ventana de contexto.

Por ejemplo, `AI_AGG(reviews, 'Resumir las reseñas de libros en 200 palabras')` devolverá un resumen de los comentarios de los usuarios.

A diferencia de `COMPLETE (SNOWFLAKE.CORTEX)` y `SUMMARIZE (SNOWFLAKE.CORTEX)`, esta función admite conjuntos de datos mayores que la ventana de contexto máxima del modelo de lenguaje.

### Sintaxis

```sql
AI_AGG( <expr>, <task_description> )
```

### Argumentos

- `<expr>` :(Obligatorio): Esta expresión contiene texto sobre el que se realizará una operación de agregación, como reseñas de restaurantes o transcripciones telefónicas.
- `<task_description>`: Un string que contiene una descripción en lenguaje natural de la agregación que se realizará, por ejemplo, «Resumir las reseñas» o «Identificar a todas las personas mencionadas y escribir una breve biografía de cada una».

### Notas de uso

Para un rendimiento óptimo, debemos seguir estas pautas:

- Utilizar texto en inglés sencillo para la descripción de la tarea.
- Describa el texto proporcionado en la descripción de la tarea. Por ejemplo, en lugar de una descripción como "resumir", utilice "Resumir las transcripciones de las llamadas telefónicas".
- Describa el caso de uso previsto. Por ejemplo, en lugar de "encontrar la mejor reseña", utilice "Encontrar la reseña de restaurante más positiva y mejor escrita para destacarla en el sitio web del restaurante".
- Considere dividir la descripción de la tarea en varios pasos. Por ejemplo, en lugar de "Resumir los nuevos artículos", utilice "Recibirá artículos de noticias de diversas editoriales que presentan eventos desde diferentes perspectivas. Por favor, cree un resumen conciso y detallado de los textos fuente sin omitir ninguna información crucial".

## Ejemplos

`AI_AGG` se puede utilizar como una función escalar simple en strings constantes.

In [ ]:
SELECT AI_AGG('[Excelente, Excelente, Maravilloso, Mediocre]',
              'Resume las calificaciones del producto para una publicación de blog dirigida a los consumidores. Develve la respuesta en Español.');

`AI_AGG` se puede utilizar en una columna de datos.

En el siguiente ejemplo utilizaremos la función sobre la tabla `reviews` que previamente hemos creado.

In [ ]:
SELECT AI_AGG(review,
              'Resume en tres puntos las calificaciones de los parques de diversiones para una presentación ejecutiva. Devuelve la respuesta en Español.')
  FROM curso_ia.seccion_2.reviews;

`AI_AGG` también se puede utilizar en combinación con `GROUP BY`. Para mostrar este ejemplo vamos a crear una tabla de nombre `transcripciones` y agregarle algunos datos.

In [ ]:
-- Crear la tabla de transcripciones
CREATE TABLE curso_ia.seccion_2.transcripciones (
    id_cliente STRING,
    transcripcion_llamada STRING,
    sentimiento_bandera STRING
);

In [ ]:
-- Insertar los valores en la tabla
INSERT INTO curso_ia.seccion_2.transcripciones (id_cliente, transcripcion_llamada, sentimiento_bandera) VALUES
('CLI_001', 'Buenos días, quería agradecer por el excelente servicio que recibí ayer. El técnico fue muy profesional y resolvió mi problema rápidamente. Estoy muy satisfecho con la atención. Definitivamente los recomendaré a mis amigos.', 'P'),

('CLI_002', 'Estoy muy molesto porque es la tercera vez que llamo por el mismo problema y nadie me da una solución definitiva. He perdido mucho tiempo esperando y esto es inaceptable. Quiero hablar con un supervisor inmediatamente.', 'N'),

('CLI_003', 'Hola, solo llamaba para consultar sobre mi facturación. Vi algunos cargos que no entiendo muy bien. ¿Podrían explicarme qué incluye cada concepto? Gracias por su ayuda, han sido muy amables al atenderme.', 'P'),

('CLI_004', 'El producto que compré no funciona como esperaba y cuando intenté devolverlo me dijeron que ya no se puede. Esto es una estafa total. Nunca más voy a comprar con ustedes y voy a dejar reseñas negativas en todas partes.', 'N'),

('CLI_005', 'Perfecto, muchas gracias por la información. Me ha quedado todo muy claro sobre el nuevo plan de servicios. Procederé con la actualización como me sugirieron. El proceso fue muy sencillo y el agente fue súper paciente conmigo.', 'P');

In [ ]:
SELECT * FROM curso_ia.seccion_2.transcripciones;

Ahora utilizamos `AI_AGG` sobre la tabla creada utilizando un `GRUOP BY`

In [ ]:
SELECT sentimiento_bandera,
       AI_AGG(transcripcion_llamada, 'De estas transcripciones de llamadas de varios clientes, resumen el sentimiento de las llamadas sin omitir datos importantes.') AS transcripcion_resumida
  FROM curso_ia.seccion_2.transcripciones
 GROUP BY 1;

La descripción de la tarea se puede utilizar para diversas tareas de agregación y para configurar el estilo y el tono de la respuesta, veamos un ejemplo.

In [ ]:
SELECT sentimiento_bandera,
       AI_AGG(transcripcion_llamada, 'De estas transcripciones de llamadas de varios clientes, identifica en una sola palabra el sentimiento más positivo y tradúcelo a Inglés y Francés. La respuesta solo debe contener el sentimiento en ambos idiomas.') AS transcripcion_resumida
  FROM curso_ia.seccion_2.transcripciones
 GROUP BY 1;

## Función **`AI_SUMMARIZE_AGG`**

Esta función resume una columna de texto. Por ejemplo, `AI_SUMMARIZE_AGG(churn_reason)` devolverá un resumen de la columna `churn_reason`.

A diferencia de `AI_COMPLETE` y `SUMMARIZE (SNOWFLAKE.CORTEX)`, esta función admite conjuntos de datos mayores que la ventana de contexto máxima del modelo de lenguaje.

## Sintaxis

```sql
AI_SUMMARIZE_AGG( <expr> )
```

## Notas de uso

Esta función proporciona un resumen general. Para un resumen más específico, debemos utilizar `AI_AGG`.

## Ejemplo

In [ ]:
SELECT AI_SUMMARIZE_AGG(transcripcion_llamada) AS resumen
FROM curso_ia.seccion_2.transcripciones;

`AI_SUMMARIZE_AGG` también se puede utilizar en combinación con `GROUP BY.`

In [ ]:
SELECT sentimiento_bandera,
       AI_SUMMARIZE_AGG(transcripcion_llamada) AS resumen
FROM curso_ia.seccion_2.transcripciones
GROUP BY 1;